In [ ]:
# ===========================================================================
# DIANI Mamoudou Sékou
#
# Juin 2022
#===========================================================================


In [ ]:
# Importation des bibliothèques nécessaires
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from keras.layers import Conv2D, MaxPooling2D, Input, BatchNormalization, UpSampling2D, Activation, Dropout, Flatten, Dense
from keras.callbacks import CSVLogger, ModelCheckpoint, EarlyStopping
import tensorflow as tf
from keras import backend as K
import matplotlib.pyplot as plt
import time

# Configuration des GPUs et CPUs
config = tf.compat.v1.ConfigProto(device_count={'GPU': 2, 'CPU': 4})
sess = tf.compat.v1.Session(config=config)
tf.compat.v1.keras.backend.set_session(sess)

In [ ]:
# Declaration des chemins vers les fichiers cibles

# Le dossier principal qui contient les donnees
mainDataPath = "donnees2/"

# Le dossier contenant les images d'entrainement
trainPath = mainDataPath + "entrainement"

# Le dossier contenant les images de validation
validationPath = mainDataPath + "validation"

# Le dossier contenant les images de test
testPath = mainDataPath + "test"

# Le nom du fichier du modele a sauvegarder
modelsPath = "Model.hdf5"

In [ ]:
# Declaration des variable 
training_batch_size = 18000  
validation_batch_size = 6000  

# Configuration des  images 
image_scale = 100 # la taille des images
image_channels = 3  # le nombre de canaux de couleurs 
images_color_mode = "rgb"  
image_shape = (image_scale, image_scale, image_channels) # la forme des images d'entrees

# Configuration des parametres d'entrainement
fit_batch_size = 180 # le nombre d'images entrainees ensemble: un batch
fit_epochs = 70 # Le nombre d'epoques 


In [ ]:

# ==========================================
# ==========CHARGEMENT DES IMAGES===========
# ==========================================
# Charge les donnees d'entrainement en memoire
training_data_generator = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True, validation_split=0.25)

# Charge les donnees de validation en memoire
validation_data_generator = ImageDataGenerator(rescale=1. / 255)

# training_generator: indique la methode de chargement des donnees d'entrainement
training_generator = training_data_generator.flow_from_directory(
    trainPath, # Place des images d'entrainement
    color_mode=images_color_mode, # couleur des images
    target_size=(image_scale, image_scale),# taille des images
    batch_size=training_batch_size, # nombre d'images a entrainer 
    class_mode="categorical", # classement
    shuffle=True, 
    subset='training') # on "brasse" (shuffle) les donnees pour prevenir le surapprentissage

# validation_generator: indique la methode de chargement des donnees de validation
validation_generator = training_data_generator.flow_from_directory(
    trainPath, # Place des images de validation
    color_mode=images_color_mode, # couleur des images
    target_size=(image_scale, image_scale),  # taille des images
    batch_size=validation_batch_size,  # nombre d'images a valider
    class_mode="categorical",  # classement 
    shuffle=True, 
    subset='validation') # on "brasse" (shuffle) les donnees pour prevenir le surapprentissage


# On charge les donnees d'entrainement et de validation
(x_train, y_train) = training_generator.next()
(x_val, y_val) = validation_generator.next()

# On Normalise les images en les divisant par la plus grande pixel dans les images (generalement c'est 255)
max_value = float(x_train.max())
x_train = x_train.astype('float32') / max_value
x_val = x_val.astype('float32') / max_value

In [ ]:
# Couche d'entree
input_layer = Input(shape=image_shape)

In [ ]:
# Partie feature extraction 
def feature_extraction(input):    
    x = Conv2D(100, (3, 3), padding='same')(input) 
    x = Activation("relu")(x)
    x = Conv2D(100, (3, 3), padding='same')(x) 
    x = Activation("relu")(x)
    x = MaxPooling2D((2, 2), padding='same', strides=(2,2))(x)
    x = BatchNormalization()(x)
    
    x = Conv2D(128, (3, 3), padding='same')(x) 
    x = Activation("relu")(x)
    x = Conv2D(128, (3, 3), padding='same')(x) 
    x = Activation("relu")(x)
    x = Conv2D(128, (3, 3), padding='same')(x) 
    x = Activation("relu")(x)
    x = MaxPooling2D((2, 2), padding='same', strides=(2,2))(x)
    x = BatchNormalization()(x)

    x = Conv2D(256, (3, 3), padding='same')(x) 
    x = Activation("relu")(x)
    x = Conv2D(256, (3, 3), padding='same')(x) 
    x = Activation("relu")(x)
    x = Conv2D(256, (3, 3), padding='same')(x) 
    x = Activation("relu")(x)
    x = Conv2D(256, (3, 3), padding='same')(x) 
    x = Activation("relu")(x)
    x = MaxPooling2D((2, 2), padding='same', strides=(2,2))(x)
    x = BatchNormalization()(x)

    x = Conv2D(256, (3, 3), padding='same')(x) 
    x = Activation("relu")(x)
    x = Conv2D(256, (3, 3), padding='same')(x) 
    x = Activation("relu")(x)
    x = Conv2D(256, (3, 3), padding='same')(x) 
    x = Activation("relu")(x)
    x = Conv2D(256, (3, 3), padding='same')(x) 
    x = Activation("relu")(x)
    x = MaxPooling2D((2, 2), padding='same', strides=(2,2))(x)
    x = BatchNormalization()(x)

    x = Conv2D(512, (3, 3), padding='same')(x) 
    x = Activation("relu")(x)
    x = Conv2D(512, (3, 3), padding='same')(x) 
    x = Activation("relu")(x)
    x = Conv2D(512, (3, 3), padding='same')(x) 
    x = Activation("relu")(x)
    x = Conv2D(512, (3, 3), padding='same')(x) 
    x = Activation("relu")(x)
    x = MaxPooling2D((2, 2), padding='same', strides=(2,2))(x)
    x = BatchNormalization()(x)

    x = Conv2D(512, (3, 3), padding='same')(x) 
    x = Activation("relu")(x)
    x = Conv2D(512, (3, 3), padding='same')(x) 
    x = Activation("relu")(x)
    x = Conv2D(512, (3, 3), padding='same')(x) 
    x = Activation("relu")(x)
    x = Conv2D(512, (3, 3), padding='same')(x) 
    x = Activation("relu")(x)
    encoded = MaxPooling2D((2, 2), padding='same',strides=(2,2))(x)
    return encoded


# Partie completement connect�e (Fully Connected Layer)
def fully_connected(encoded):
    x = Flatten(input_shape=image_shape)(encoded)
    x = Dense(512)(x)
    x = Activation("relu")(x)
    x= Dropout(0.5)(x)

    x = Dense(512)(x)
    x = Activation("relu")(x)
    x= Dropout(0.5)(x)
    x = Dense(6)(x)
    sortie = Activation('softmax')(x)
    return sortie

# Declaration du modele:
model = Model(img_input, fully_connected(feature_extraction(input_layer)))

# Affichage des parametres du modele
model.summary()
# Compilation du modele en definissant la fonction de perte, l'optimisateur et la valeur afficher durant l'entrainement
model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])

In [ ]:
# Sauvegarde du modèle ayant le meilleur val_accuracy pendant l'entraînement dans Model.hdf5. 
modelcheckpoint = ModelCheckpoint(filepath=modelsPath,
                                  monitor='val_accuracy', verbose=1, save_best_only=True, mode='auto')


# entrainement du modele
classifier = model.fit(x_train, y_train,
                       epochs=fit_epochs, # nombre d'�poques
                       batch_size=fit_batch_size, # nombre d'images entrain�es ensemble
                       validation_data=(x_val, y_val), # donn�es de validation
                       verbose=1, # mets cette valeur � 0, si vous voulez ne pas afficher les d�tails d'entrainement
                       callbacks=[modelcheckpoint], # les fonctions � appeler � la fin de chaque �poque (dans ce cas modelcheckpoint: qui sauvegarde le mod�le)
                       shuffle=True)# shuffle les images

# ==========================================
# ========AFFICHAGE DES RESULTATS===========
# ==========================================

# Plot accuracy over epochs (precision par epoque)
print(classifier.history.keys())
plt.plot(classifier.history['accuracy'])
plt.plot(classifier.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'])
fig = plt.gcf()
plt.show() 

In [ ]:
#Affichage de la courbe de perte (loss curve)
print(classifier.history.keys())
plt.plot(classifier.history['loss'])
plt.plot(classifier.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'])
fig = plt.gcf()
plt.show()